In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sys


if __name__ == '__main__':
    np.random.seed(123)
    tf.random.set_seed(123)
    csv_input=pd.read_csv("2019BAPTC.csv")
    length_of_sequences=len(csv_input)
    maxlen=230
    pred_date=1
    data_input=pd.DataFrame(csv_input)
    x_data=pd.DataFrame({"LETTUCES-Green Ice-":data_input["LETTUCES-Green Ice-"],
                         "BROCCOLI-wonderball-Untrimmed":data_input["BROCCOLI-wonderball-Untrimmed"],
                         "CELERY-Lollo Rossa-":data_input["CELERY-Lollo Rossa-"],
                         "temperature":data_input["temperature"],
                         "raifall":data_input["raifall"]})
    t_data=pd.DataFrame({"LETTUCES-Green Ice-":data_input["LETTUCES-Green Ice-"]})
    x_data=x_data.interpolate()
    t_data=t_data.interpolate()
    x=[]
    t=[]


    for i in range(length_of_sequences - maxlen - pred_date):
        x.append(np.array(x_data).tolist()[i: i+maxlen])
        t.append(np.array(t_data).tolist()[i+maxlen+pred_date])

    x_preds=x

    t= np.array(t).reshape(-1, 1).astype(np.float32)
    x = np.array(x).reshape(-1, maxlen, 5).astype(np.float32)
    x_train, x_val, t_train, t_val = \
        train_test_split(x, t, test_size=0.2, shuffle=False)

    
    for i in range(length_of_sequences - maxlen - pred_date, length_of_sequences - maxlen + 1):
        x_preds.append(np.array(x_data).tolist()[i: i+maxlen])
    x_preds=np.array(x_preds).reshape(-1, maxlen, 5).astype(np.float32)
    model = Sequential()
    model.add(LSTM(200, activation='tanh',
                        recurrent_activation='sigmoid',
                        kernel_initializer='glorot_normal',
                        recurrent_initializer='orthogonal'))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.Adam(learning_rate=0.01,
                                beta_1=0.9, beta_2=0.999, amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error')
    es = EarlyStopping(monitor='val_loss',
                       patience=70,
                       verbose=1)
    hist = model.fit(x_train, t_train,
                     epochs=500, batch_size=100,
                     verbose=2,
                     validation_data=(x_val, t_val),
                     callbacks=[es])
    model.save("F:\\Official purpose\\Crop_price_prediction.h5")
    #sin = toy_problem(T, ampl=0.)
    gen = [None for i in range(maxlen)]

    
    for i in range(length_of_sequences - maxlen - pred_date):
        z=x[:1+i]
        t1=np.array(t).reshape(-1,1)
        preds=model.predict(z[-1:])
        print("i=", i)
        print("t=", t1[i,0])
        print("preds=", preds[0,0])

        
    for i in range(length_of_sequences - maxlen - pred_date, length_of_sequences - maxlen +1):
        z_preds=x_preds[:1+i]
        preds=model.predict(z_preds[-1:])
        print("i_preds=", i)
        print(preds[0,0])


    for i in range(length_of_sequences - maxlen - pred_date):
        preds = model.predict(z[-1:])
        print(z[-1:])
        print(preds)
        z = np.append(z, preds)[1:]
        z = z.reshape(-1, maxlen, 5)
        gen.append(preds[0, 0])
        print(z)
        print(t[i+maxlen+pred_date])
    loss, acc=model.evaluate(x_val,t_val,verbose=0)
    print("test_loss=", loss, "test_acc=", acc)

Epoch 1/500
1/1 - 4s - loss: 0.4344 - val_loss: 1.8330
Epoch 2/500
1/1 - 0s - loss: 4.9585 - val_loss: 0.0381
Epoch 3/500
1/1 - 0s - loss: 0.2119 - val_loss: 0.0269
Epoch 4/500
1/1 - 0s - loss: 0.3414 - val_loss: 0.0355
Epoch 5/500
1/1 - 0s - loss: 0.3103 - val_loss: 0.0431
Epoch 6/500
1/1 - 0s - loss: 0.2004 - val_loss: 0.0931
Epoch 7/500
1/1 - 0s - loss: 0.1201 - val_loss: 0.1201
Epoch 8/500
1/1 - 0s - loss: 0.2086 - val_loss: 0.0306
Epoch 9/500
1/1 - 0s - loss: 0.0405 - val_loss: 0.0315
Epoch 10/500
1/1 - 1s - loss: 0.1089 - val_loss: 0.0665
Epoch 11/500
1/1 - 1s - loss: 0.1169 - val_loss: 0.1082
Epoch 12/500
1/1 - 1s - loss: 0.0768 - val_loss: 0.1304
Epoch 13/500
1/1 - 1s - loss: 0.0372 - val_loss: 0.1034
Epoch 14/500
1/1 - 1s - loss: 0.0476 - val_loss: 0.0812
Epoch 15/500
1/1 - 1s - loss: 0.0631 - val_loss: 0.0752
Epoch 16/500
1/1 - 0s - loss: 0.0517 - val_loss: 0.0711
Epoch 17/500
1/1 - 1s - loss: 0.0472 - val_loss: 0.0668
Epoch 18/500
1/1 - 1s - loss: 0.0475 - val_loss: 0.3193
E

IndexError: index 231 is out of bounds for axis 0 with size 44